In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install -q transformers accelerate torch torchvision pillow

from transformers import AutoTokenizer, AutoModelForVision2Seq
from PIL import Image
import torch

# Model name (corrected to a valid vision-language model)
model_name = "Qwen/Qwen2-VL-7B-Instruct"  # Updated to correct model name

try:
    # Load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    model = AutoModelForVision2Seq.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True
    )

    # Check if CUDA is available, else use CPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    # Load and preprocess image
    image_path = "/kaggle/input/truckairport/atx-cargo-box-airport.jpg"
    try:
        image = Image.open(image_path).convert("RGB")
    except FileNotFoundError:
        print(f"Error: Image file not found at {image_path}")
        exit(1)

    # Prepare multimodal input for the model
    inputs = tokenizer(
        text="Describe this image in detail.",
        images=[image],  # Pass the actual image object
        return_tensors="pt"
    ).to(device)

    # Generate caption
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=200,
            do_sample=False  # For deterministic output
        )

    # Decode the generated tokens
    caption = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    print("Generated Caption:\n", caption)

except Exception as e:
    print(f"An error occurred: {str(e)}")